In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
from collections import Counter

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [15]:
#loading data
headlines = pd.read_csv('FiQA_train_ABSA_financial_headlines.tsv',delimiter='\t',header=0)
posts = pd.read_csv('FiQA_train_ABSA_financial_posts.tsv',delimiter='\t',header=0)
posts = posts.drop(['Unnamed: 0', 'id'],axis=1)
headlines = headlines.drop(['Unnamed: 0', 'id'],axis=1)
posts_text = posts.iloc[:,:1]
posts_score = posts.iloc[:,1:]
headlines_text = headlines.iloc[:,:1]
headlines_score = headlines.iloc[:,1:]
text_frames = [posts_text,headlines_text]


,text
0,Cuadrilla files to delay application to frack ...
1,Royal Mail chairman Donald Brydon set to step ...
2,UPDATE 2-EasyJet sees better first half on low...
3,Slump in Weir leads FTSE down from record high
4,AstraZeneca wins FDA approval for key new lung...
5,Sainsbury chief warns of squeeze on high stree...
6,UPDATE 1-Lloyds to cut 945 jobs as part of 3-y...
7,Primark plots expansion in US and Europe
8,Standard Chartered Shifts Emerging-Markets Str...
9,Barclays fined for anti-money-laundering failings


In [3]:
#counting word frequency to create vocabulary
counter_ = Counter()
for review in reviwes.values:
    for word in review[0].split(" "):
        counter_[word]+=1
# print(counter_)

In [4]:
print(len(counter_))

74074


In [5]:
#keeping only 10000 most commmon words, frequency of 10000th word can be used to see if we need ot increase or decrease this number
vocabulary = sorted(counter_,key=counter_.get,reverse = True)[:10000]

In [6]:
#word to index, will need this mapping to create word vector 
word_to_index = dict()
for index,word in enumerate(vocabulary):
    word_to_index[word] = index

In [7]:
def get_word_vector(data):
    vector = np.zeros(len(vocabulary), dtype=np.int_)
    for word in data.split(' '):
        index = word_to_index.get(word, None)
        if index != None:
            vector[index]+=1
    return vector

In [8]:
review_vectors = np.zeros((len(reviwes),len(vocabulary)),dtype=np.int_)
for index,(_,data) in enumerate(reviwes.iterrows()):
    review_vectors[index] = get_word_vector(data[0])

In [9]:
review_vectors[:5]

array([[ 18,   9,  27, ...,   0,   0,   0],
       [  5,   4,   8, ...,   0,   0,   0],
       [ 78,  24,  12, ...,   0,   0,   0],
       [167,  53,  23, ...,   0,   0,   0],
       [ 19,  10,  11, ...,   0,   0,   0]])

In [25]:
Y = (labels=='positive').astype(np.int_)
shuffle = np.arange(len(labels))
np.random.shuffle(shuffle)
test_train_split = 0.9
train_split, test_split = shuffle[:int(len(labels)*test_train_split)], shuffle[int(len(labels)*test_train_split):]
trainX, trainY = review_vectors[train_split,:], to_categorical(Y.values[train_split], 2)
testX, testY = review_vectors[test_split,:], to_categorical(Y.values[test_split], 2)
trainX = tflearn.data_utils.pad_sequences(trainX, maxlen=100, value=0.)
testX = tflearn.data_utils.pad_sequences(testX, maxlen=100, value=0.)

In [26]:
# Network building
tf.reset_default_graph()
net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

In [27]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY,n_epoch=2,validation_set=0.1, show_metric=True,batch_size=32)

Training Step: 1265  | total loss: 1.38629 | time: 113.180s
| Adam | epoch: 002 | loss: 1.38629 - acc: 0.4944 -- iter: 20224/20250
Training Step: 1266  | total loss: 1.38629 | time: 116.991s
| Adam | epoch: 002 | loss: 1.38629 - acc: 0.5012 | val_loss: 1.38629 - val_acc: 0.9969 -- iter: 20250/20250
--


In [28]:
# (np.array(model.predict(testX))[:,0]>=0.5).astype(np.int_)

predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
print("Test accuracy: ", test_accuracy)

Test accuracy:  0.9976
